## Running the Supervised Optimizer

From start to finish, on pretrained weights


In [1]:
import torch
import numpy as np
import os
from Hamiltonian import Ising
from model import TransformerModel
from optimizer_supervised import Optimizer

In [2]:
system_sizes = np.arange(4, 21, 2).reshape(-1, 1)
Hamiltonians = [Ising(size, periodic=True) for size in system_sizes]
param_dim = Hamiltonians[0].param_dim
embedding_size = 32
n_head = 8
n_hid = embedding_size
n_layers = 8
dropout = 0
minibatch = 10000
param_range = None
point_of_interest = None
use_SR = False

print("Sizes:\n", system_sizes)
print("Hamiltonians:", Hamiltonians)
print("Dimensions of parameter space:", param_dim)
print("Number of units in a feedforward layer:", n_hid)

Sizes:
 [[ 4]
 [ 6]
 [ 8]
 [10]
 [12]
 [14]
 [16]
 [18]
 [20]]
Hamiltonians: [<Hamiltonian.Ising object at 0x7ce28fd92420>, <Hamiltonian.Ising object at 0x7ce444400500>, <Hamiltonian.Ising object at 0x7ce444402d50>, <Hamiltonian.Ising object at 0x7ce28f9ad250>, <Hamiltonian.Ising object at 0x7ce28f9ac800>, <Hamiltonian.Ising object at 0x7ce28f9ad2b0>, <Hamiltonian.Ising object at 0x7ce28f9ad310>, <Hamiltonian.Ising object at 0x7ce28f9ad370>, <Hamiltonian.Ising object at 0x7ce28f9ad3d0>]
Dimensions of parameter space: 1
Number of units in a feedforward layer: 32


/home/spandan/Projects/tqs/Hamiltonian_utils.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  system_size = torch.tensor(system_size, dtype=torch.int64).reshape(-1)


In [3]:
testmodel = TransformerModel(
    system_sizes,
    param_dim,
    embedding_size,
    n_head,
    n_hid,
    n_layers,
    dropout=dropout,
    minibatch=minibatch,
)

results_dir = "results"
paper_checkpoint_name = "ckpt_100000_Ising_32_8_8_0.ckpt"
paper_checkpoint_path = os.path.join(results_dir, paper_checkpoint_name)
checkpoint = torch.load(paper_checkpoint_path)
testmodel.load_state_dict(checkpoint)

/home/spandan/anaconda3/envs/tqs2/lib/python3.12/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer was not TransformerEncoderLayer
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


<All keys matched successfully>

In [4]:
opt = Optimizer(testmodel, Hamiltonians, point_of_interest=point_of_interest)

In [5]:
epochs=10
param_range=torch.tensor([[0.5], [1.5]])
param_step=torch.tensor([0.1])

opt.train(
    epochs=10,
    param_range=param_range,
    param_step=t0.1
)

TypeError: 'float' object is not iterable